In [138]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Dependencies

In [139]:
from craigslist import CraigslistHousing
import timeit, os
from tqdm import tqdm
import pickle


import requests
from selenium import webdriver
from pymongo import MongoClient
import time
import pandas as pd
import missingno
import re
# import json
# from pprint import pprint
from datetime import datetime, timedelta, timezone

from urls_list import * #where all urls and paths are saved
from config import *
# from harvesine_distance import harvesine_distance

In [140]:
## Craigslist provides a total of 3000 listings at once.

In [141]:
def craigs_list_api_call():
# This function returns the result of the listings based on site and category
# The search URL is as below
# https://toronto.craigslist.org/search/tor/apa?
    cl_tor_housing = CraigslistHousing(site='toronto',
                                       area='tor',
                                       category='apa',
                             filters={'bundle_duplicates': 1})

    #If geotagged=True, the results will include the (lat, lng) in the 'geotag' attrib (this will make the process a little bit longer).
    craiglist_housing = []

    start = timeit.timeit()
    for result in tqdm(cl_tor_housing.get_results(sort_by='newest', geotagged=True)):
        craiglist_housing.append(result)
    end = timeit.timeit()
    print(f"Time elapsed is {end - start}")
    return craiglist_housing

craiglist_housing = craigs_list_api_call()

3000it [13:49,  3.62it/s]

Time elapsed is 0.007837696000933647


In [214]:
def differencer(DF):
    #Finds out the data to be scraped after comparing with the 
    #already existing data in the data base
    # Input - DF
    # Output - To be scraped IDs
    # Also clears the CurrentRental collection and updates it with the current rental
    all_postings_ids = DF['id'].map(lambda x: 'c_'+str(x))
    client = MongoClient(db_connection_string)
    db = client.ETLInsights #DB
    retrieved_postings = list(db.HistoricRental.find({'id': {'$in': list(all_postings_ids)}}))
    to_be_scraped_ids = set(all_postings_ids).difference(map(lambda x: x['id'], retrieved_postings))
    #Truncate and update the CurrentRental collection
    db.CurrentRental.delete_many({})
    db.CurrentRental.insert_many(retrieved_postings)
    client.close()
    return list(map(lambda x: x[2:], to_be_scraped_ids))

In [215]:
to_be_scraped_ids = differencer(pd.DataFrame(craiglist_housing))

In [17]:
# with open(os.path.join('..','Stage','craiglist_housing.pkl'), 'wb') as f:
#     pickle.dump(craiglist_housing, f)

# with open(os.path.join('..','Stage','craiglist_housing.pkl'), 'rb') as f:
#     craiglist_housing = pickle.load(f)

###  Scraping the craigslist posting data using the urls obtained through craigslist Python Module call

In [216]:
def instatiate_driver():
    #########################################################################################
    #Instatiate Selenium driver
    #Returns the handle object
    #########################################################################################
    chrome_options = webdriver.ChromeOptions()
    CHROMEDRIVER_PATH = executable_path
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, options=chrome_options)
    return driver

In [217]:
def craigs_list_scrape(craigs_list_post_docs):
    #Instatiate the selenium driver
    driver = instatiate_driver()
    error_list = []
    

    craigs_list_post_docs_cp = craigs_list_post_docs.copy()
    for post in tqdm(craigs_list_post_docs_cp):

        #Visit the url
        driver.get(post['url'])

        #Separate Try Except to handle each cases separately
        #Apartment feature(Some extra feature beside the title)
        try:
            apartment_feature = driver.find_element_by_css_selector('span[class="postingtitletext"] span[class="housing"]').text
        except:
            #print(post['id'])
            error_list.append({post['id']:'apartment_feature'})
            apartment_feature = None
        finally:
            post['apartment_feature'] = apartment_feature

        #First image if present   
        try:
            image = None
            if post['has_image']:
                image = driver.find_element_by_css_selector('div[class="gallery"]').find_element_by_css_selector('img').get_attribute('src')
        except:
            print(post['id'])
            error_list.append({post['id']:'image'})
        finally:
            post['has_image'] = image

        #Body of the post  
        try:
            posting = driver.find_element_by_id("postingbody").text
        except:
            print(post['id'])
            error_list.append({post['id']:'postingbody'})
            posting = None
        finally:
            post['posting'] = posting

        #Attributes      
        try:
            attributes = [elem.text for elem in driver.find_elements_by_css_selector('p[class="attrgroup"]  span')]
        except:
            print(post['id'])
            error_list.append({post['id']:'attributes'})
            attributes = []
        finally:
            post['attributes'] = attributes
            
        time.sleep(1)
            
    driver.quit()       
    return craigs_list_post_docs_cp

In [225]:
#Filter out the to be scraped listings and scrape further
TBS = list(filter(lambda x: x['id'] in to_be_scraped_ids, craiglist_housing))
craiglist_housing_enriched = craigs_list_scrape(TBS)

 82%|████████▏ | 145/176 [03:45<00:46,  1.51s/it]

7200656895


100%|██████████| 176/176 [04:35<00:00,  1.57s/it]


# Transform

### Initial form of data after scraping

In [229]:
DF = pd.DataFrame(craiglist_housing_enriched)
DF.head()

,id,repost_of,name,url,datetime,last_updated,price,where,has_image,geotag,deleted,apartment_feature,posting,attributes
0,7201022350,None,Very large & bright 1 bedroom apartment,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-22 15:06,2020-09-22 15:06,"$1,700",Oriole Gardens,https://images.craigslist.org/00L0L_8M4hjG7Hfx...,"(43.691097, -79.396414)",False,/ 1br - 650ft2 -,6 months lease for a very large and bright 1-b...,"[1BR / 1Ba, 650ft2, available oct 1, apartment..."
1,7201017028,None,QUEEN W/LISGAR-MAIN FLOOR-2 BED/2 BATH/1 PARKI...,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-22 14:59,2020-09-22 14:59,"$3,500",TORONTO,https://images.craigslist.org/00R0R_fQkDyE7Nkd...,"(43.641286, -79.422871)",False,/ 2br -,$3500 Utilities included!\n\nMust have full cr...,"[2BR / 2Ba, available now, duplex, w/d in unit..."
2,7201014800,None,Unfurnished Peaceful Loft (1 Bed+ 1 Den),https://toronto.craigslist.org/tor/apa/d/downt...,2020-09-22 14:58,2020-09-22 14:58,"$3,300",Spadina & Richmond,https://images.craigslist.org/01111_aZVbazuG6u...,"(43.6404, -79.3995)",False,/ 1br - 1000ft2 -,1000 sqf+ Loft\n- Balcony overlooking CN tower...,"[1BR / 1Ba, 1000ft2, available dec 1, loft, w/..."
3,7201012159,None,DANFORTH/DONALNDS-3 BED/2 BATH-GLEBEHOLME AVE,https://toronto.craigslist.org/tor/apa/d/east-...,2020-09-22 14:53,2020-09-22 14:53,"$3,800",TORONTO,https://images.craigslist.org/00404_2kLt6BEdNj...,"(43.6872, -79.3368)",False,/ 3br -,$3800 +Utilities\n\nMust have full credit repo...,"[3BR / 2Ba, available now, duplex, w/d in unit..."
4,7201009453,None,Furnished Peaceful Loft (1bed+den),https://toronto.craigslist.org/tor/apa/d/downt...,2020-09-22 14:49,2020-09-22 14:49,"$3,400",Richmond & Spadina - Downtown,https://images.craigslist.org/01111_aZVbazuG6u...,"(43.6404, -79.3995)",False,/ 1br - 1000ft2 -,1000 sqf+ Furnished Loft\n- Balcony overlookin...,"[1BR / 1Ba, 1000ft2, available dec 1, furnishe..."


### Remove Duplicates and unreliable data
- All duplicated data are removed
- Dont consider empty postings
- Drop all those rows which don't have both 'geotag' and 'where'
- Convert attributes list to string
- Convert the apartment feature none to ''(string)

In [230]:
def clean_craigslist(DF):
    ### Remove Duplicates and unreliable data
    # - All duplicated data are removed
    # - Dont consider empty postings
    # - Drop all those rows which don't have both 'geotag' and 'where'
    # Convert attributes list to string
    # Convert the apartment feature none to ''(string)

    #Duplicates cannot hash list
    DF.drop_duplicates(subset=DF.columns[:-1], keep="first", inplace=True)
    #Dont consider empty postings
    DF.dropna(subset=['posting'], inplace=True)
    #Location:- Coming to whereabouts, in most of the cases, we have geotag. Some of the cases which doesn't have geotag,
    #we have "where". Check below.We need to drop all those rows which don't have both
    DF.dropna(subset=['where', 'geotag'], how='all', inplace=True)
    #Convert attributes list to string
    DF['attributes'] = DF['attributes'].map(lambda x: ' '.join(x))
    #Convert the apartment feature none to ''(string)
    DF['apartment_feature'].fillna(value=' ', inplace=True)#A string val
    #Combine the "attributes", "apartment_feature", "posting" together as "text" column
    DF["text"] = DF.apply(lambda x: x["attributes"]+x["apartment_feature"]+x["posting"], axis=1)
    #Use DF["text"] for all further extractions 
    return DF


In [261]:
DF = clean_craigslist(pd.DataFrame(craiglist_housing_enriched))

## Deive new Features
- Combine the "attributes", "apartment_feature", "posting" together as "text" column for string search using regex to derive new features
- New features derived are
    - sf - Square Feet : (int or None) Based on ft2|SqFt in the text data
    - br - Bed Room: (float or None) Number of bedrooms, based on what is preceding BR 
    - ba - Bath: (float or None) Number of bath, based on what is preceding Ba
    - cats_allowed - True or False, based on the presence of 'cats are OK - purrr'
    - dogs are OK - wooof - True or False, based on the presence of 'dogs are OK - wooof'
    - Type - Type of the housing 1 out of ``['condo', 'house', 'apartment', 'suite', 'townhouse', 'loft', 'duplex',
       'flat', 'cottage', 'land']`` or ``None``
    - furnished - True/Flase/None - based on the presense of (un|non)(-) furnished strings or nothing
    

```unique_attributes = set()
for entry in DF['attributes']:
    unique_attributes = unique_attributes.union(set(entry))
unique_attributes```  

In [262]:
def extract(s):
    #DF['Text'] comes here as the input
    #Square Feet
    sf_found = re.findall('(\d+) *(ft2|SqFt)', s)
    sf =  (None if not sf_found else int(sf_found[0][0]))

    #Bed Room
    br_found = re.findall('(\d){0,1}(.5){0,1} *BR', s)
    br = float(''.join(br_found[0])) if  br_found and ''.join(br_found[0]) else None #To handle ('','','') situation
            
    
    #Bath
    ba_found = re.findall('(\d){0,1}(.5){0,1}(\+){0,1} *B[aA]',s)
    ba = float(re.sub('[^\d]', '', ''.join(ba_found[0]))) if  ba_found and re.sub('[^\d]', '', ''.join(ba_found[0])) else None
    
    
    #'cats are OK - purrr'
    cats_allowed = (True if re.findall('cats are OK - purrr', s) else False)
    
    #'dogs are OK - wooof'
    dogs_allowed = (True if re.findall('dogs are OK - wooof', s) else False)
    
    #Check the type of the commodity
    re.sub(r'[\n\.,!/?()]', ' ', s.lower()) #Remove unnecessary chars. We need to capture apartment\ also as word apartment
    cleaned_s = re.sub('[\n\\.,!\/?]', ' ', s)
    found_type = re.findall(r"\b(townhouse|loft|land|house|flat|duplex|condo|cottage|suite)\b", cleaned_s, flags=re.IGNORECASE) 
    Type = (None if not found_type else found_type[0].lower())
    #Though apartment is a generic term, some people mention the type as apartment, we are going to take that as the last priority
    #if nothing else is mentioned
    if not Type:
        found_type = re.findall(r"\bapartment\b", cleaned_s, flags=re.IGNORECASE)
        Type = (None if not found_type else found_type[0].lower())
        
    #Furnished or Unfurnished checks
    found_un = re.findall('(non|un)-*(?=furnished)', s , flags=re.IGNORECASE)
    furnished = False
    if not found_un:
        #furnished = None #Nothing found
        found_furnished = re.findall('furnished', s , flags=re.IGNORECASE)
        furnished = (None if not found_furnished else True)
    
    return [sf,br,ba, cats_allowed, dogs_allowed, Type, furnished]

In [263]:
DF[['sf', 'BR', 'Ba','cats_allowed', 'dogs_allowed', 'Type', 'furnished']] = pd.DataFrame(DF['text'].map(lambda x : extract(x)).to_list(), index=DF.index)

### Geocode the address to lat long

In [264]:
#Dictionary with index and address
def geocode(addresses):
    #Input: a dictionary with key as index and value as address
    #Output: a dictionary with key as index and value as (lat, long) tuple
    lat_long = {}
    for index in addresses:
        try:
            url = geocode_api+f'?address={addresses[index]}'+f'&key={google_api}'
            url = re.sub(' +', '%20', url)
            response = requests.get(url)
            if response.ok:
                content = response.json()
                lat_long[index]=tuple(content['results'][0]['geometry']['location'].values())
            else:
                lat_long[index]=None
        except Exception as e:
            print(e)
            lat_long[index]=None
    return lat_long

In [265]:
addresses = DF[DF['geotag'].isnull()]['where'].to_dict()

```lat_long = geocode(addresses)```

In [266]:
lat_long = geocode(addresses)
## Replace the Null geocodes with the geocodes retrieved from the address
DF.loc[lat_long.keys(), 'geotag'] = DF.loc[lat_long.keys()].index.map(lat_long)

In [267]:
#Now no missing values in geocode
#Check for nan
missingdata_cols = DF.columns[DF.isnull().any()].to_list()
missingdata_cols

['repost_of', 'where', 'has_image', 'sf', 'BR', 'Ba', 'furnished']

### Postal codes based on lat long

In [268]:
#https://maps.googleapis.com/maps/api/geocode/json?latlng=40.714224,-73.961452&key=YOUR_API_KEY
#reverse_geocode_api = 'https://maps.googleapis.com/maps/api/geocode/json'

#Dictionary with index and address
def reverse_geocode(loc):
    #Input: tuple - (lat, long)
    #Output: string - Postal Code
    lat, long = loc
    postal_code = None
    try:
        url = reverse_geocode_api+f'?latlng={lat},{long}&result_type=postal_code'+f'&key={google_api}'
        url = re.sub(' +', '%20', url)
        #print(url)
        response = requests.get(url)
        if response.ok:
            content = response.json()
            postal_code = content['results'][0]["address_components"][0]['long_name'] if 'postal_code' in content['results'][0]\
["address_components"][0]['types'] else None
    except Exception as e:
        print(e)
    return postal_code   

In [269]:
DF['postal_code'] = DF['geotag'].map(lambda x: reverse_geocode(x))
DF[DF['postal_code'].notnull()].shape
DF.shape
# Discarding those 6 items which doesn't have postal code
DF.drop(index=DF[DF['postal_code'].isnull()].index, inplace=True)

list index out of range
list index out of range


(171, 23)

(173, 23)

``DF['postal_code'] = DF['geotag'].map(lambda x: reverse_geocode(x))``

``DF[DF['postal_code'].notnull()].shape``

``DF.shape``

``#Discarding those 7 items which doesn't have postal code``

``DF.drop(index=DF[DF['postal_code'].isnull()].index, inplace=True)``

In [270]:
## Some Florida postal codes in the dataset. Remove them
#DF['postal_code'][DF['postal_code'].map(lambda x : re.findall('^[\d].*', x)[0] if re.findall('^[\d].*', x) else None).notnull()]  


In [271]:
Florida_Zipcodes = DF['postal_code'][DF['postal_code'].map(lambda x : re.findall('^[\d].*', x)[0] if re.findall('^[\d].*', x) else None).notnull()].index    
#DF.drop(index=Florida_Zipcodes, inplace=True)

In [272]:
DF.drop(index=Florida_Zipcodes, inplace=True)

### Save standalone craigslist dataset

In [54]:
with open(os.path.join('..','Stage','craiglist_standalone.pkl'), 'wb') as f:
    pickle.dump(DF, f)

In [55]:
with open(os.path.join('..','Stage','craiglist_standalone.pkl'), 'rb') as f:
    DF = pickle.load(f)

### Tranformation required to append the craigslist table with kijii table

- **``We need to concatenate craigslist table with kijiji table``**
- **``All the column names have to be unique``**

In [273]:
def clean_rental_for_merge(df):
    DF= df.copy()
    #To rename
    DF.rename(columns={"has_image":"image", "name":"title", "datetime":"post_published_date", "where":"address", "Type":"rental_type", "BR":"bedrooms", "Ba":"bathrooms", "sf":"sqft", "text":"description"}, inplace=True)              
    #To extract
    DF['pet_friendly'] = DF.apply(lambda x: x['cats_allowed'] | x['dogs_allowed'], axis=1)
    DF['lat'] = DF.geotag.map(lambda x: float(x[0]))
    DF['long'] = DF.geotag.map(lambda x: float(x[1]))
    #To clean
    DF['price'] = DF['price'].map(lambda x: re.sub('[^\d]', '',x))
    #To drop
    DF.drop(["repost_of", "deleted", "apartment_feature", "cats_allowed", "dogs_allowed", "geotag", "posting", "attributes", "last_updated"], axis=1, inplace=True)
    #Add one source column
    DF['source'] = 'craigslist'
    #FSA
    DF['FSA']=DF['postal_code'].map(lambda x:x.split(' ')[0])
    #Reorder
    DF = DF[['id', 'title', 'price', 'sqft','image','url','post_published_date', 'lat', 'long', 'postal_code', 'FSA', 'rental_type','bedrooms', 'bathrooms', 'furnished', 'pet_friendly', 'description', 'source']]
    #Append c_ to the index
    DF['id'] = DF['id'].map(lambda x: 'c_'+str(x))
    #Change date
    DF.post_published_date = DF.post_published_date.map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M").strftime("%Y-%m-%d"))
    
    return DF
    

In [274]:
DF = clean_rental_for_merge(DF)

In [275]:
DF

,id,title,price,sqft,image,url,post_published_date,lat,long,postal_code,FSA,rental_type,bedrooms,bathrooms,furnished,pet_friendly,description,source
0,c_7201022350,Very large & bright 1 bedroom apartment,1700,650.0,https://images.craigslist.org/00L0L_8M4hjG7Hfx...,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-22,43.691097,-79.396414,M4V 1V7,M4V,house,1.0,1.0,False,False,1BR / 1Ba 650ft2 available oct 1 apartment w/d...,craigslist
1,c_7201017028,QUEEN W/LISGAR-MAIN FLOOR-2 BED/2 BATH/1 PARKI...,3500,NaN,https://images.craigslist.org/00R0R_fQkDyE7Nkd...,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-22,43.641286,-79.422871,M6J,M6J,duplex,2.0,2.0,None,False,2BR / 2Ba available now duplex w/d in unit det...,craigslist
2,c_7201014800,Unfurnished Peaceful Loft (1 Bed+ 1 Den),3300,1000.0,https://images.craigslist.org/01111_aZVbazuG6u...,https://toronto.craigslist.org/tor/apa/d/downt...,2020-09-22,43.640400,-79.399500,M5V 0E4,M5V,loft,1.0,1.0,False,False,1BR / 1Ba 1000ft2 available dec 1 loft w/d in ...,craigslist
3,c_7201012159,DANFORTH/DONALNDS-3 BED/2 BATH-GLEBEHOLME AVE,3800,NaN,https://images.craigslist.org/00404_2kLt6BEdNj...,https://toronto.craigslist.org/tor/apa/d/east-...,2020-09-22,43.687200,-79.336800,M4J 3Y4,M4J,duplex,3.0,2.0,None,False,3BR / 2Ba available now duplex w/d in unit no ...,craigslist
4,c_7201009453,Furnished Peaceful Loft (1bed+den),3400,1000.0,https://images.craigslist.org/01111_aZVbazuG6u...,https://toronto.craigslist.org/tor/apa/d/downt...,2020-09-22,43.640400,-79.399500,M5V 0E4,M5V,loft,1.0,1.0,True,False,1BR / 1Ba 1000ft2 available dec 1 furnished lo...,craigslist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,c_7192901469,"HUGE BACHELOR-FULL KITCHEN & 4 PIECE BTHRM, HU...",1250,NaN,https://images.craigslist.org/00a0a_dN2sUXBESE...,https://toronto.craigslist.org/tor/apa/d/west-...,2020-09-09,43.665500,-79.437800,M6H 4B9,M6H,apartment,0.0,1.0,None,False,0BR / 1Ba available oct 1 apartment LARGE BACH...,craigslist
172,c_7192591951,"HUGE BACHELOR-FULL KITCHEN & 4 PIECE BTHRM, HU...",1250,NaN,https://images.craigslist.org/00a0a_dN2sUXBESE...,https://toronto.craigslist.org/tor/apa/d/west-...,2020-09-08,43.665500,-79.437800,M6H 4B9,M6H,apartment,0.0,1.0,None,False,0BR / 1Ba available oct 1 apartment LARGE BACH...,craigslist
173,c_7192590559,"1 BDRM MAIN FLOOR, LARGE LIVING ROOM, HUGE CLO...",1700,NaN,https://images.craigslist.org/00g0g_i7qHOtjsDd...,https://toronto.craigslist.org/tor/apa/d/west-...,2020-09-08,43.665500,-79.437800,M6H 4B9,M6H,apartment,1.0,1.0,None,False,1BR / 1Ba available oct 1 apartment/ 1br -10 M...,craigslist
174,c_7191794235,"1 BDRM MAIN FLOOR, LARGE LIVING ROOM, HUGE CLO...",1700,NaN,https://images.craigslist.org/00g0g_i7qHOtjsDd...,https://toronto.craigslist.org/tor/apa/d/west-...,2020-09-07,43.665500,-79.437800,M6H 4B9,M6H,apartment,1.0,1.0,None,False,1BR / 1Ba available oct 1 apartment/ 1br -10 M...,craigslist


## Save all dataframes as csv

In [63]:
DF.to_csv(os.path.join('..','Output','Rental_Craigslist.csv'), index=False)

In [64]:
DF

,id,title,price,sqft,image,url,post_published_date,lat,long,postal_code,FSA,rental_type,bedrooms,bathrooms,furnished,pet_friendly,description,source
0,c_7200612095,1 BR condo at 68 Abell St--Queen West-- Availa...,1600,420.0,https://images.craigslist.org/00J0J_8zBmjxmDi0...,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-21,43.642777,-79.424243,M6J,M6J,condo,1.0,1.0,None,False,1BR / 1Ba 420ft2 available oct 1 condo w/d in ...,craigslist
1,c_7187385635,ALL INCLUSIVE 2 bedroom Furnished -Main Subway...,1595,NaN,https://images.craigslist.org/01010_fAb2gD7xgA...,https://toronto.craigslist.org/tor/apa/d/east-...,2020-09-21,43.693118,-79.297176,M4C 5B9,M4C,house,2.0,1.0,True,False,2BR / 1Ba available oct 1 furnished house w/d ...,craigslist
2,c_7200606976,Beautiful Rooms for rent in North York (immedi...,600,NaN,https://images.craigslist.org/00x0x_1wSiPFUT3f...,https://toronto.craigslist.org/tor/apa/d/north...,2020-09-21,43.804775,-79.380681,M2M 2Z5,M2M,house,4.0,3.0,True,False,4BR / 3Ba available now furnished house laundr...,craigslist
3,c_7200610375,GORGEOUS RENOVATED-2 BEDROOM UNIT- QUEENSWAY &...,2100,950.0,https://images.craigslist.org/00B0B_83hUDfsiTP...,https://toronto.craigslist.org/tor/apa/d/etobi...,2020-09-21,43.633793,-79.487718,M8Y 1R8,M8Y,suite,2.0,1.0,None,True,2BR / 1Ba 950ft2 available oct 1 cats are OK -...,craigslist
4,c_7200602760,Basement Apartment,750,NaN,https://images.craigslist.org/00j0j_gntmBwGe9O...,https://toronto.craigslist.org/tor/apa/d/richm...,2020-09-21,43.866900,-79.441400,L4C 9S5,L4C,apartment,NaN,NaN,None,False,available sep 25 apartment w/d in unit A beaut...,craigslist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,c_7175105221,"1BDR Condo above Loblaws, Queen/Portland",2200,650.0,https://images.craigslist.org/00t0t_gyX609atyM...,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-05,43.647085,-79.401455,M5V 1Y9,M5V,condo,1.0,1.0,None,False,1BR / 1Ba 650ft2 available oct 1 condo w/d in ...,craigslist
2996,c_7190663212,"Bloor and Spadina ANNEX, Downtown, mins to U o...",2700,NaN,https://images.craigslist.org/00808_8JF9DKqTva...,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-05,43.665900,-79.405425,M5S 2K8,M5S,house,3.0,2.0,None,False,3BR / 2Ba apartment w/d in unit no smoking no ...,craigslist
2997,c_7188127944,Bright Condo for Rent - 1Bd + Den + 1.5 Br in ...,2220,665.0,https://images.craigslist.org/00s0s_3MjkAuvQtj...,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-05,43.654895,-79.378975,M5B 1T8,M5B,condo,1.0,15.0,None,False,1BR / 1.5Ba 665ft2 available now condo w/d in ...,craigslist
2998,c_7173283683,1 Bed & 1 Bath + Parking in Yorkville,1999,NaN,https://images.craigslist.org/00v0v_hFIddf4dId...,https://toronto.craigslist.org/tor/apa/d/toron...,2020-09-05,43.672503,-79.388764,M5R 1E9,M5R,condo,1.0,1.0,None,False,1BR / 1Ba available oct 1 condo w/d in unit no...,craigslist


# Load

In [125]:
from pymongo import MongoClient

In [276]:
client = MongoClient(db_connection_string)
client.ETLInsights.HistoricRental.insert_many(DF.T.to_dict().values())
client.ETLInsights.CurrentRental.insert_many(DF.T.to_dict().values())
client.close()